In [6]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [3]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [4]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Sarah Johnson',
  'subject': 'Introductory Psychology',
  'stars': 4,
  'review': 'Dr. Johnson is an excellent professor who really cares about her students. Her lectures are engaging and she is always available to help during office hours.'},
 {'professor': 'Professor Michael Lee',
  'subject': 'Calculus I',
  'stars': 3,
  'review': "Professor Lee's lectures can be dry at times, but he is very knowledgeable and tests fairly. The workload is manageable if you stay on top of the material."},
 {'professor': 'Dr. Elizabeth Rodriguez',
  'subject': 'Introduction to Biology',
  'stars': 5,
  'review': 'Dr. Rodriguez is an incredibly passionate and effective teacher. Her labs are well-designed and help cement the concepts from lecture. I would highly recommend taking any of her classes.'},
 {'professor': 'Professor David Wilson',
  'subject': 'American History Since 1865',
  'stars': 4,
  'review': 'Professor Wilson is a knowledgeable and engaging lecturer. His classes a

In [7]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review['professor'],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [8]:
# Insert the embeddings into the Pinecone index
index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 21 Aug 2024 06:46:12 GMT', 'Content-Type': 'application/json', 'Content-Length': '74', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '41', 'x-pinecone-request-id': '342828487321042409', 'x-envoy-upstream-service-time': '42', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"No vectors provided for upsert request","details":[]}


In [ ]:
print(f"Upserted count: {upsert_response['upserted_count']}")

# Print index statistics
print(index.describe_index_stats())